# Hierarchical Community Detection
## In this notebook:
* We run the following algorithm on each of the six networks. The algorithm:

1. takes the graph G and samples it by removing half of the edges (or nodes) and extracting the GCC (let's call it wcc).
2. runs hierarchical clustering algorithm paris on wcc and extract the dendrogram Z.
3. cuts Z at 10 different heights (from 2 to 12 communities) and evaluates modularity for each of the 10 partitions.
4. given the set of modularities it looks for the index of the 'jump'. We have a 'jump' in modularity when the value i+1 is at least 10% greater than value i.
5. selects as best partition the partition of the index of the 'jump'.
6. removes nodes belonging to small communities (i.e. communities with size < 100).
7. At this point, what we have is: a set of nodes (wcc nodes. The ones sampled at the beginning) and a best_partition (a set of labels assigning each node to a community).
8. steps from 1. to 6. are repeated 100 times. So we get 100 different set of nodes (from 10 independent samples of the same network) and 100 different best_partitions.

* Then we plot experimental RMCA distribution. We define RMCA (Rate of the Most Common Assignment) as the normalized occurrence of the most common label of a certain node.

## Importing networks

In [ ]:
import networkx as nx
import pandas as pd
import numpy as np

In [ ]:
t = pd.read_csv('/../github_alltime_edgelist.csv').drop(['Unnamed: 0'],axis=1)
t[t['weight'] > 1] #applying filter on edges' weight

In [ ]:
usrs_net_df0 = t[t['time_window'] == 'i']
usrs_net_df1 = t[t['time_window'] == 'ii']
usrs_net_df2 = t[t['time_window'] == 'iii']
usrs_net_df3 = t[t['time_window'] == 'iv']
usrs_net_df4 = t[t['time_window'] == 'v']
usrs_net_df5 = t[t['time_window'] == 'vi']

In [ ]:
del t

In [ ]:
#I apply the filter on the weight. I discard links with weight = 1
G0w = nx.from_pandas_edgelist(usrs_net_df0[usrs_net_df0['weight'] > 1], source='user1', target='user2', edge_attr=True, create_using=nx.DiGraph())
G1w = nx.from_pandas_edgelist(usrs_net_df1[usrs_net_df1['weight'] > 1], source='user1', target='user2', edge_attr=True, create_using=nx.DiGraph())
G2w = nx.from_pandas_edgelist(usrs_net_df2[usrs_net_df2['weight'] > 1], source='user1', target='user2', edge_attr=True, create_using=nx.DiGraph())
G3w = nx.from_pandas_edgelist(usrs_net_df3[usrs_net_df3['weight'] > 1], source='user1', target='user2', edge_attr=True, create_using=nx.DiGraph())
G4w = nx.from_pandas_edgelist(usrs_net_df4[usrs_net_df4['weight'] > 1], source='user1', target='user2', edge_attr=True, create_using=nx.DiGraph())
G5w = nx.from_pandas_edgelist(usrs_net_df5[usrs_net_df5['weight'] > 1], source='user1', target='user2', edge_attr=True, create_using=nx.DiGraph())

In [ ]:
del G0w,G1w,G2w,G3w,G4w,G5w

In [ ]:
#extracting the GCC out of the network obtained by the application of the filter weight > 1
wcc0_w = max(nx.weakly_connected_components(G0w), key=len) 
wcc0_w = G0w.subgraph(wcc0_w).copy()

wcc1_w = max(nx.weakly_connected_components(G1w), key=len) 
wcc1_w = G1w.subgraph(wcc1_w).copy()

wcc2_w = max(nx.weakly_connected_components(G2w), key=len) 
wcc2_w = G2w.subgraph(wcc2_w).copy()

wcc3_w = max(nx.weakly_connected_components(G3w), key=len) 
wcc3_w = G3w.subgraph(wcc3_w).copy()

wcc4_w = max(nx.weakly_connected_components(G4w), key=len) 
wcc4_w = G4w.subgraph(wcc4_w).copy()

wcc5_w = max(nx.weakly_connected_components(G5w), key=len) 
wcc5_w = G5w.subgraph(wcc5_w).copy()

In [ ]:
N0wcc_w, N1wcc_w, N2wcc_w, N3wcc_w, N4wcc_w, N5wcc_w = wcc0_w.number_of_nodes(), wcc1_w.number_of_nodes(), wcc2_w.number_of_nodes(), wcc3_w.number_of_nodes(), wcc4_w.number_of_nodes(), wcc5_w.number_of_nodes()
n0wcc_w, n1wcc_w, n2wcc_w, n3wcc_w, n4wcc_w, n5wcc_w = wcc0_w.number_of_edges(), wcc1_w.number_of_edges(), wcc2_w.number_of_edges(), wcc3_w.number_of_edges(), wcc4_w.number_of_edges(), wcc5_w.number_of_edges()

print('i   -> Number of nodes:', N0wcc_w, ', number of edges', n0wcc_w)
print('ii  -> Number of nodes:', N1wcc_w, ', number of edges', n1wcc_w)
print('iii -> Number of nodes:', N2wcc_w, ', number of edges', n2wcc_w)
print('iv  -> Number of nodes:', N3wcc_w, ', number of edges', n3wcc_w)
print('v   -> Number of nodes:', N4wcc_w, ', number of edges', n4wcc_w)
print('vi  -> Number of nodes:', N5wcc_w, ', number of edges', n5wcc_w)

In [ ]:
#total number of tweets in the preCOVID retweet network 
s = 0
for e in wcc0_w.edges:
    s+=wcc0_w.get_edge_data(*e)['weight']
s    

In [ ]:
s = 0
for e in wcc1_w.edges:
    s+=wcc1_w.get_edge_data(*e)['weight']
s    

In [ ]:
s = 0
for e in wcc2_w.edges:
    s+=wcc2_w.get_edge_data(*e)['weight']
s    

In [ ]:
s = 0
for e in wcc3_w.edges:
    s+=wcc3_w.get_edge_data(*e)['weight']
s    

In [ ]:
s = 0
for e in wcc4_w.edges:
    s+=wcc4_w.get_edge_data(*e)['weight']
s    

In [ ]:
s = 0
for e in wcc5_w.edges:
    s+=wcc5_w.get_edge_data(*e)['weight']
s    

In [ ]:
wcc1_w['3327831430']['882965185237065728']['weight']

In [ ]:
#applying natural log to the weights of all the links
for e in wcc0_w.edges:
    wcc0_w.get_edge_data(*e)['weight'] = np.log(wcc0_w.get_edge_data(*e)['weight'])
for e in wcc1_w.edges:
    wcc1_w.get_edge_data(*e)['weight'] = np.log(wcc1_w.get_edge_data(*e)['weight'])
for e in wcc2_w.edges:
    wcc2_w.get_edge_data(*e)['weight'] = np.log(wcc2_w.get_edge_data(*e)['weight'])
for e in wcc3_w.edges:
    wcc3_w.get_edge_data(*e)['weight'] = np.log(wcc3_w.get_edge_data(*e)['weight'])
for e in wcc4_w.edges:
    wcc4_w.get_edge_data(*e)['weight'] = np.log(wcc4_w.get_edge_data(*e)['weight'])
for e in wcc5_w.edges:
    wcc5_w.get_edge_data(*e)['weight'] = np.log(wcc5_w.get_edge_data(*e)['weight'])

In [ ]:
wcc1_w['3327831430']['882965185237065728']['weight']

## Hierarchical clustering

In [ ]:
from sknetwork.hierarchy import Paris, Ward
from scipy.cluster.hierarchy import dendrogram, fcluster
import random
from sklearn.metrics.cluster import adjusted_rand_score
import networkx.algorithms.community as nx_comm
from itertools import combinations
from statistics import mean
from statistics import stdev
import collections
import sys
import matplotlib.pyplot as plt
import csv
from sklearn.metrics import confusion_matrix

## Defining functions

In [ ]:
def get_partition(clustering_array, G): #clustering_array represents a certain partition of the network G
    #clustering array is a numpy array containing the indices of the communities to which each node is associated. 
    #For example, if there are just 2 communities, clustering_array is going to be something 
    #like [1,1,2,1,2, ... ,2,2,1,2]. So the first node goes into community 1, so does the second, 
    #the third node goes into community 2, etc...
    #G is the network of which I want to get a partition
    series = pd.Series(clustering_array)
    series.index = G.nodes
    communities = np.unique(clustering_array) #gets all the different communities
    partition = []
    for c in communities:
        ind = series[series == c].index #indices assigned to the community c
        partition.append(ind)
    
    return partition #it is something like [indices1, indices2, ...]

In [ ]:
#given a list, this function returns the index of the value after the 'jump'
#by 'jump' we mean a sudden increase of the values of the list
#the jump must be at least perc% to be considered jump

def jump_detector(l, hc_alg, perc, min_val = 0.4):
    s = pd.Series(l)
    #print(s)
    opt = 0
    for i in s.index:
        if s[i] < min_val: #sometimes modularity starts from zero, increases to ~0.4 and then has another jump to
            #~0.5, so we want to keep the second jump and not the first one.
            continue
        if hc_alg == 'ward':
            if s[i+1] < s[i]:
                opt = i
                return opt

            elif (s[i+1]-s[i])/s[i] > perc: #if finds a suitable big enough jump the function returns the index of the jump
                
                opt = i + 1
                return opt
            
        elif hc_alg == 'paris':
            
            if (s[i+1]-s[i])/s[i] > perc: #if finds a suitable big enough jump the function returns the index of the jump
                
                opt = i + 1
                return opt

        if i==len(s)-2 and opt==0:#if gets to the end without finding a jump it takes the index of the minimum 
            #value grater than the threshold min_val

            s = s[s>min_val]

            opt = min(s.index)
            return opt
            
    return opt

In [ ]:
#returns the modularities associated to each partition present in clustering_arrays
def get_modularities(clustering_arrays,G):
    mod_array = []
    for c_a in clustering_arrays:
        partition = get_partition(c_a,G)
        mod = nx_comm.modularity(G,partition)
        mod_array.append(mod)
    
    return mod_array

#here is where the hierarchical clustering algorithm runs on GCC of the sampled network
#then the dendrogram is cut at 10 different heights and the corresponding clustering_arrays are put together
#then we evaluate the modularity of each partition (ie clustering_array)
#then we select the index of the jump
def dendrogram_and_jumpidx(G,hc_alg, min_val, perc):
    if hc_alg == 'paris':
        paris_ward = Paris()
    
    elif hc_alg == 'ward':
        paris_ward = Ward()
        
    else:
        print(hc_alg,': unknown clustering algorithm')
        return
    
    adj_m = nx.adjacency_matrix(G)
    Z0 = paris_ward.fit_transform(adj_m)
    

    clust_arrays = []
    for ti in range(2,11,1):    
        clust_array = fcluster(Z=Z0,t=ti,criterion='maxclust')
        clust_arrays.append(clust_array)

    
    mods = get_modularities(clust_arrays,G)
    jd = jump_detector(mods,hc_alg=hc_alg, min_val=min_val, perc= perc)
    

    
    cab = fcluster(Z=Z0,t= jd+2, criterion='maxclust') #clust_array_best
    
    return cab ,jd, mods

In [ ]:
#removes half random links from a network, then returns the GCC of the remaning network
def remove_edges(g, fraction): #with fraction=2 we remove half of the links
    G = g.copy()
    n = G.number_of_edges()
    to_remove=random.sample(G.edges(), k=int(n/fraction))
    G.remove_edges_from(to_remove)
    
    wcc = max(nx.weakly_connected_components(G), key=len) 
    wcc = G.subgraph(wcc).copy()

    return wcc

def remove_nodes(g, fraction): #with fraction=2 we remove half of the links
    G = g.copy()
    N = G.number_of_nodes()
    to_remove=random.sample(G.nodes(), k=int(N/fraction))
    G.remove_nodes_from(to_remove)
    
    wcc = max(nx.weakly_connected_components(G), key=len) 
    wcc = G.subgraph(wcc).copy()

    return wcc

In [ ]:
#takes as keys the nodes of a certain GCC and as values the list containing the label of the community to which
#each node belongs to. thr is the threshold for small communities. All the communities smaller than threshold are 
#removed (ie all the nodes belonging to the small community are removed).
def remove_small_communities(keys, values, thr):
    label_dict = dict(zip(keys, values)) #creates a dict with keys: nodes, values: community label
    labels = label_dict.values() #takes the list of the labels
    
    bins = np.bincount(list(labels))[1:] #evaluates how many nodes there are in each community
    
    to_remove = []
    for i in range(len(bins)): #append the labels to remove to a list
        if bins[i] < thr:
            to_remove.append(i+1)
            
    for l in to_remove: #removes the labels present in the list to_remove
        label_dict = dict((k, v) for k, v in label_dict.items() if v != l)
    
    #after removing small communities
    #we want the community to be ordered by size (ie the smallest community is always labeled 1 and the biggest
    #is always labeled N, where N is the number of communities.)
    ll = np.arange(1,len(set(label_dict.values())) + 1, 1) #it is just an array like [1,2, ... , N]
    
    #we want set(dict.values) to be equal to ll, because sometimes we have for example 4 communities and the third 
    #is 'small' then it is removed and so set(dict.values) is {1,2,4} when we want it to be {1,2,3}.
    chs = changes_to_make(list(set(label_dict.values())),list(ll))#gets the changes to transform set(dict.values)
    #into [1,2,...,N]
    
    s = pd.Series(data=label_dict.values(), index=label_dict.keys())#turns the dict into a series to use 'replace'
    
    for ch in chs: #renames communities to be renamed
        s = s.replace(ch[0],'x')
        s = s.replace(ch[1],'y')
            
        s = s.replace('x', ch[1])
        s = s.replace('y', ch[0])
            
    
    label_dict = s.to_dict() #back to dict
    

    return label_dict

In [ ]:
#takes to lists and returns the changes (ie the switchs of elements) to make l equal to true_l
def changes_to_make(l,true_l):
    s = pd.Series(l)
    changes = []

    while list(s) != true_l:

        chs = [(i,j) for i,j in zip(list(s),true_l)]
        for i in range(len(chs)):
            if chs[i][0] == chs[i][1]:
                continue
            
            change = chs[i]

            s = s.replace(change[0],'x')
            s = s.replace(change[1],'y')
            
            s = s.replace('x', change[1])
            s = s.replace('y', change[0])
            
            changes.append(change)
            break
            
    return changes

In [ ]:
#takes a list of labels [1,1,2,4,5,2,1,...] and a list of changes to make. For example we may want to switch
#(2,1) or (1,3) etc.
#returns the list of labels with the changes made.
def lab_adjuster(labels,changes):
    
    s = pd.Series(labels) 
    
    for change in changes: #renames community according to changes
        s = s.replace(change[0],'x')
        s = s.replace(change[1],'y')
            
        s = s.replace('x', change[1])
        s = s.replace('y', change[0])
    
    return list(s)

In [ ]:
#every tuple contains (nodes of the sample, ordered label). The labels are now ordered by community size.
#so if a node belongs to the smallest community it is labeled 1, if it belongs to the biggest community it is 
#labeled N.
#returns a dictionary like: keys: nodes, values: list containing all the labels of that nodes in different samples.
def nodes_dictionary(tuples):
    nodes_dict = {}
    for tup in tuples:
        for node, label in zip(tup[0],tup[1]):

            nodes_dict.setdefault(node,[])
            nodes_dict[node].append(label)
            
    return nodes_dict

In [ ]:
#takes two different sets of nodes of the same network which was sampled twice
#and two clust_arrays containing the assignment to the community of the nodes in each set of nodes
#finds common nodes and puts into a df each common node (index) and the two different assignment from the two 
#different partitions

def common_nodes_label(nodes1,nodes2,clust_array1, clust_array2):
    
    zip_iterator1 = zip(nodes1, clust_array1)
    zip_iterator2 = zip(nodes2, clust_array2)
    
    dict1 = dict(zip_iterator1)
    dict2 = dict(zip_iterator2)
    
    common_nodes = list(set(nodes1).intersection(set(nodes2)))
    
    new_dict1 = {your_key: dict1[your_key] for your_key in common_nodes}
    new_dict2 = {your_key: dict2[your_key] for your_key in common_nodes}
    
    df = pd.DataFrame(index=common_nodes)
    df['d1'] = new_dict1.values()
    df['d2'] = new_dict2.values()
    
    return df

In [ ]:
#returns the sum of non-diagonal elements of a matrix
def sum_non_diagonal(mat):
    rw, cl = mat.shape
    dia = np.diag_indices(min(rw,cl)) #indices of diagonal elements
    dia_sum = sum(mat[dia]) # sum of diagonal elements
    off_dia_sum = np.sum(mat) - dia_sum # subtract the diagonal sum from total array sum
    return off_dia_sum

In [ ]:
#swaps two rows (x,y) of a given matrix a
def swap_rows(a,x,y):
    a[[x, y]] = a[[y, x]]
    return a

#takes a matrix and returns the couples of rows to be swapped in order to make that matrix diagonal
def changes_to_make2(mx):
    changes = []
    
    rows, columns = mx.shape
    
    if sum_non_diagonal(mx) == 0.:
        return changes
    
    else:
        while sum_non_diagonal(mx) != 0.:
            for r in range(rows):
                am = np.argmax(mx[r]) #argmax of the row r
                if am != r:
                    changes.append((r+1,am+1))
                    mx = swap_rows(mx,am,r)
                    if sum_non_diagonal(mx) == 0.:
                        return changes
                
                else:
                    continue
                        
    return changes

In [ ]:
#takes a confusion matrix and returns the changes to make in order to make the matrix have the larger value of each 
#row on the diagonal
def go_through_cm(cm):
    
    if cm.shape != cm[~np.all(cm == 0, axis=1)].shape:
        print('!')
    
    cm = cm[~np.all(cm == 0, axis=1)] #removing rows (or columns with all zeroes)
    
    cm = cm.transpose()
    
    if cm.shape != cm[~np.all(cm == 0, axis=1)].shape:
        print('!!')
    
    cm = cm[~np.all(cm == 0, axis=1)]
    
    cm = cm.transpose()
    
    rows, columns = cm.shape
    
    I = np.zeros(rows*columns).reshape(rows,columns)
    max_ind = cm.argmax(1) #list saying what is the index of the max in each row
    
    for i in range(len(max_ind)): #creates I with 1 in the argmax position and zero elsewhere
        
        ind = max_ind[i]
        
        I[i][ind] = 1

    I = I.transpose() #so that I have to switch rows and not columns
    return changes_to_make2(I)

In [ ]:
#given a network it samples the network, does Paris hierarchical clustering, assigns a label to the nodes 
#of the sample. Repeats this for N times and return a dictionary having keys: nodes' id and values: a list
#containing the labels of the nodes. The length of the lists is going to be <= N.
def partitions_dict2(g, fraction=2, edges=True, N=100, hc_alg = 'paris', min_val = .4, perc=0.1, thr=100):
    
    tuples = []
    
    for i in range(N): #we want N different samples of the network
        sys.stdout.write(f'\r{i+1}/{N}')
        
        if edges:
            wcc = remove_edges(g,fraction)
        else:
            wcc = remove_nodes(g,fraction)
        
        
        clust_array_best, _, _ = dendrogram_and_jumpidx(wcc, hc_alg, min_val, perc=perc)#for each of the samples 
        #we run paris and obtain the dendrogram (Z0). We obtain the clust_array_best (ie the partition after the 
        #jump in modularity) 
        #we also get the jump_index to keep track of it
        
        #removes the nodes belonging to small communities (ie community size < 100)
        
        dic = remove_small_communities(wcc.nodes(),clust_array_best, thr=thr)
        
        nodes, labels = list(dic.keys()), list(dic.values())
        
        if i == 0:
            ref_dict = dic
        print(np.bincount(list(dic.values()))[1:], len(np.bincount(list(dic.values()))[1:]))
        if len(np.bincount(list(dic.values()))[1:]) < 2:
            print('continue')
            continue

        
        df = common_nodes_label(ref_dict.keys(),dic.keys(),ref_dict.values(),dic.values())
        
        cm = confusion_matrix(df['d1'],df['d2'])
        
        changes = go_through_cm(cm)
        labels_adj = lab_adjuster(labels,changes)
        
        #we add to tuples the tuple (nodes of the sampled network,best assignment to communities of these nodes)
        tuples.append((nodes,labels_adj))
        
    return nodes_dictionary(tuples)

In [ ]:
#given a network it samples the nwtwork, does Paris hierarchical clustering, assigns a label to the nodes 
#of the sample. Repeats this for N times and return a dictionary having keys: nodes' id and values: a list
#containing the labels of the nodes. The length of the lists is going to be <= N.
def partitions_dict5(g, fraction=2, edges=True, N=100, hc_alg = 'paris', min_val = .4, perc=0.1, thr=100):
    
    tuples = []
    i=0
    while i < N: #we want N different samples of the network
        sys.stdout.write(f'\r{i+1}/{N}')
        
        if edges:
            wcc = remove_edges(g,fraction)
        else:
            wcc = remove_nodes(g,fraction)
        
        
        clust_array_best, _, _ = dendrogram_and_jumpidx(wcc, hc_alg, min_val, perc=perc)#for each of the samples 
        #we run paris and obtain the dendrogram (Z0). We obtain the clust_array_best (ie the partition after the 
        #jump in modularity) 
        #we also get the jump_index to keep track of it
        
        #removes the nodes belonging to small communities (ie community size < 100)
        
        dic = remove_small_communities(wcc.nodes(),clust_array_best, thr=thr)
        
        nodes, labels = list(dic.keys()), list(dic.values())
        
        if i == 0:
            ref_dict = dic
        print(np.bincount(list(dic.values()))[1:], len(np.bincount(list(dic.values()))[1:]))
        if len(np.bincount(list(dic.values()))[1:]) < 2:
            print('continue')
            continue

        
        df = common_nodes_label(ref_dict.keys(),dic.keys(),ref_dict.values(),dic.values())
        
        cm = confusion_matrix(df['d1'],df['d2'])
        
        changes = go_through_cm(cm)
        labels_adj = lab_adjuster(labels,changes)
        
        #we add to tuples the tuple (nodes of the sampled network,best assignment to communities of these nodes)
        tuples.append((nodes,labels_adj))
        
        i+=1
        
    return nodes_dictionary(tuples)

In [ ]:
def save_dict(dic, path=''):
    with open(path, 'w') as csv_file:  
        writer = csv.writer(csv_file)
        for key, value in dic.items():
            writer.writerow([key, value])

## preCOVID network

In [ ]:
%%time
#Modularity jump percentage (perc) = 0.1
part_dict0 = partitions_dict2(wcc0_w, N=100, perc=0.1)

In [ ]:
print('preCOVID network number of nodes:', wcc0_w.number_of_nodes())
print('preCOVID network covered nodes:', len(part_dict0))
print('preCOVID fraction of covered nodes:',len(part_dict0)/wcc0_w.number_of_nodes())

## earlyCOVID network

In [ ]:
%%time
part_dict1 = partitions_dict2(wcc1_w, N=100)

In [ ]:
print('earlyCOVID network number of nodes:', wcc1_w.number_of_nodes())
print('earlyCOVID network covered nodes:', len(part_dict1))
print('earlyCOVID fraction of covered nodes:',len(part_dict1)/wcc1_w.number_of_nodes())

## preVAX network

In [ ]:
%%time
#Modularity jump percentage (perc) = 0.1
part_dict2 = partitions_dict2(wcc2_w, N=10, perc=.1)

In [ ]:
print('preVAX network number of nodes:', wcc2_w.number_of_nodes())
print('preVAX network covered nodes:', len(part_dict2))
print('preVAX fraction of covered nodes:',len(part_dict2)/wcc2_w.number_of_nodes())

## earlyVAX network

In [ ]:
%%time 
part_dict3 = partitions_dict2(wcc3_w, N=100, thr=1000)

In [ ]:
print('earlyVAX network number of nodes:', wcc3_w.number_of_nodes())
print('earlyVAX network covered nodes:', len(part_dict3))
print('earlyVAX fraction of covered nodes:',len(part_dict3)/wcc3_w.number_of_nodes())

## VAXdrive network

In [ ]:
%%time
part_dict4 = partitions_dict2(wcc4_w, N=100, thr=100, min_val=.3)

In [ ]:
print('VAXdrive network number of nodes:', wcc4_w.number_of_nodes())
print('VAXdrive network covered nodes:', len(part_dict4))
print('VAXdrive fraction of covered nodes:',len(part_dict4)/wcc4_w.number_of_nodes())

## lateVAX network

In [ ]:
%%time
part_dict5 = partitions_dict5(wcc5_w, N=100, thr=1000)

In [ ]:
print('lateVAX network number of nodes:', wcc5_w.number_of_nodes())
print('lateVAX network covered nodes:', len(part_dict5))
print('lateVAX fraction of covered nodes:',len(part_dict5)/wcc5_w.number_of_nodes())

## Statistics

In [ ]:
#returns a list containing the number of times that each node was sampled (number of appereances of each node)
def number_of_appearances(dic):
    new_dict = {}
    for k in dic.keys():
        new_dict.setdefault(k,len(dic[k]))
    return list(new_dict.values())

def number_of_appearances2(dic):
    new_dict = {}
    for k in dic.keys():
        f = dic[k].split(',')
        new_dict.setdefault(k,len(f))
    return list(new_dict.values())

In [ ]:
#each node ia associated with a list of labels.
#this function returns the list of the percentage most common label of each node
#so if a node has a list like [1,1,3,1], the percentage associated will be 0.75
def rmc(dic): #RMCA
    new_dict = {}
    for k in dic.keys():
        e = len(dic[k])
        f = dic[k][1:e-1]
        
        f = f.split(', ')
        
        c = collections.Counter(f)
        x = c.most_common()[0][1]
        
        new_dict.setdefault(k,float("{0:.3f}".format(x/len(f))))
    return list(new_dict.values()), new_dict

#returns a dict with keys->nodes and values-> most common community attributed to the node
def most_common_label(dic):
    new_dict = {}
    for k in dic.keys():
        c = collections.Counter(dic[k][1:-1].split(', '))
        x = c.most_common()[0][0]
        new_dict.setdefault(k,x)
    return new_dict

In [ ]:
fig, axs = plt.subplots(2, 3, sharey=True, sharex=True, figsize=(6,4))
axs_l = axs.flatten()
ax = 10
tit= 12
hist_bins = np.linspace(0.5, 1.0, 21)

l, _ = rmc(part_dict0)
axs_l[0].hist(l, hist_bins, color='grey')
axs_l[0].set_yscale('log')
axs_l[0].grid()

l, _ = rmc(part_dict1)
axs_l[1].hist(l, hist_bins, color='grey')
axs_l[1].set_yscale('log')
axs_l[1].grid()

l, _ = rmc(part_dict2)
axs_l[2].hist(l, hist_bins, color='grey')                   
axs_l[2].set_yscale('log')
axs_l[2].grid()

l, _ = rmc(part_dict3)
axs_l[3].grid()
axs_l[3].hist(l, hist_bins, color='grey')
axs_l[3].set_yscale('log')

l, _ = rmc(part_dict4)
axs_l[4].grid()
axs_l[4].hist(l, hist_bins, color='grey')                                     
axs_l[4].set_yscale('log')

l, _ = rmc(part_dict5)
axs_l[5].grid()
axs_l[5].hist(l, hist_bins, color='grey')
axs_l[5].set_yscale('log')

axs_l[0].set_ylabel('number of nodes',fontsize=ax)
axs_l[3].set_ylabel('number of nodes',fontsize=ax)

for ax_idx in [3,4,5]:
    axs_l[ax_idx].set_xlabel('RMCA',fontsize=ax)

titles = ['i', 'ii', 'iii', 'iv', 'v', 'vi']
for i in range(len(titles)):
    axs_l[i].text(0.58, 1e4, titles[i], 
                 bbox=dict(boxstyle='round', facecolor='white', alpha=1.0))

plt.ylim(10,)
plt.tight_layout()

fig.savefig('/figures/RMCA_distribution.pdf')